In [13]:
import os
import pickle
from pathlib import Path
from pprint import pprint
from configparser import ConfigParser
from docembedder.hyperopt.utils import ModelHyperopt
from docembedder.models import BERTEmbedder
from hyperopt import space_eval
from docembedder.utils import SimulationSpecification

In [14]:
# set simulation specification
sim_spec = SimulationSpecification(
    year_start=1838,
    year_end=1950,
    window_size=25,
    debug_max_patents=200,
)

In [15]:
# read local config
config = ConfigParser()
_ = config.read("setup.ini")
hyper_fp = Path(config["DATA"]["hyper_dir"], "bert_trials.pkl")

# set general parameters
hype = ModelHyperopt(
    sim_spec=sim_spec,
    cpc_fp=Path(config["DATA"]["cpc_file"]),
    patent_dir=Path(config["DATA"]["patent_dir"]),
    trials=hyper_fp
)

In [ ]:
# run optimization
hype.optimize(label="BERT", model=BERTEmbedder, max_evals=200, n_jobs=10)

In [31]:
# display results
best_model = hype.best_model("bert", BERTEmbedder)
hype.dataframe("bert", BERTEmbedder)[:20]

,pretrained_model,loss
79,AI-Growth-Lab/PatentSBERTa,-0.128801
112,AI-Growth-Lab/PatentSBERTa,-0.128801
111,AI-Growth-Lab/PatentSBERTa,-0.128801
109,AI-Growth-Lab/PatentSBERTa,-0.128801
108,AI-Growth-Lab/PatentSBERTa,-0.128801
107,AI-Growth-Lab/PatentSBERTa,-0.128801
105,AI-Growth-Lab/PatentSBERTa,-0.128801
103,AI-Growth-Lab/PatentSBERTa,-0.128801
102,AI-Growth-Lab/PatentSBERTa,-0.128801
101,AI-Growth-Lab/PatentSBERTa,-0.128801
